In [1]:
cd ..

/Users/andrewtung/Documents/Github/latice


In [2]:
import numpy as np
from pathlib import Path
import logging
from latice.index.faiss_db import (
    FaissLatentVectorDatabase,
    FaissLatentVectorDatabaseConfig,
)
from latice.index.raw_dp_indexer import RawDiffractionPatternIndexer, RawIndexerConfig

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

In [3]:
raw_npz_path = "/Users/andrewtung/Documents/Github/latice/notebook/faiss_raw_index.npz"
pattern_path = Path("data/N=100_noised.npy")
angles_path = Path("data/anglefile_N=100.txt")

batch_size = 32
random_seed = 42
image_size = (128, 128)
top_n = 10
orientation_threshold = 3.0

raw_dimension = image_size[0] * image_size[1]
print(f"Raw pattern dimension: {raw_dimension}")

Raw pattern dimension: 16384


In [4]:
faiss_raw_db_config = FaissLatentVectorDatabaseConfig(
    npz_path=raw_npz_path, dimension=raw_dimension
)
faiss_raw_db = FaissLatentVectorDatabase(config=faiss_raw_db_config)

2025-04-25 20:26:47,262 - latice.index.faiss_db - INFO - No existing index found at /Users/andrewtung/Documents/Github/latice/notebook/faiss_raw_index.npz. Creating a new one.


In [5]:
raw_indexer_config = RawIndexerConfig(
    pattern_path=pattern_path,
    angles_path=angles_path,
    batch_size=batch_size,
    random_seed=random_seed,
    image_size=image_size,
    top_n=top_n,
    orientation_threshold=orientation_threshold,
    db_path=raw_npz_path # Ensure config uses the correct db path
)

raw_indexer = RawDiffractionPatternIndexer(
    config=raw_indexer_config,
    db=faiss_raw_db, # Pass the pre-configured DB
)

2025-04-25 20:27:19,456 - latice.index.raw_dp_indexer - INFO - Raw index dimensionality: 16384
2025-04-25 20:27:19,459 - latice.index.raw_dp_indexer - INFO - Using Faiss database with path: /Users/andrewtung/Documents/Github/latice/notebook/faiss_raw_index.npz


In [10]:
# Check if the index needs building by checking the count
if raw_indexer.db.get_count() == 0:
    print(f"Building raw FAISS index at {raw_indexer.db.npz_path}...")
    try:
        raw_indexer.build_dictionary()
        print(f"Index built successfully with {raw_indexer.db.get_count()} raw patterns.")
    except Exception as e:
        print(f"Error building dictionary: {e}")
        # Depending on the error, you might want to raise it or handle it
else:
    print(f"Raw FAISS index already exists at {raw_indexer.db.npz_path} with {raw_indexer.db.get_count()} patterns.")


Building raw FAISS index at /Users/andrewtung/Documents/Github/latice/notebook/faiss_raw_index.npz...


2025-04-25 20:33:07,595 - latice.index.raw_dp_indexer - INFO - Loading raw patterns from data/anglefile_N=100.txt


In [9]:
batch_patterns_np = np.load(pattern_path)[:5]
orientation_results_batch = raw_indexer.index_patterns_batch(
    batch_patterns_np, top_n=1, orientation_threshold=3.0
)

ValueError: This file contains pickled (object) data. If you trust the file you can load it unsafely using the `allow_pickle=` keyword argument or `pickle.load()`.

In [16]:
orientation_results_batch

[OrientationResult(query_vector=array([0., 0., 0., ..., 0., 0., 0.], shape=(16384,)), best_orientation=array([0., 0., 0.]), candidate_orientations=array([[0., 0., 0.]]), distances=array([1.0000004], dtype=float32), mean_orientation=None, success=False, similar_indices=array([0])),
 OrientationResult(query_vector=array([0., 0., 0., ..., 0., 0., 0.], shape=(16384,)), best_orientation=array([0., 1., 0.]), candidate_orientations=array([[0., 1., 0.]]), distances=array([1.0000002], dtype=float32), mean_orientation=None, success=False, similar_indices=array([0])),
 OrientationResult(query_vector=array([0., 0., 0., ..., 0., 0., 0.], shape=(16384,)), best_orientation=array([0., 2., 0.]), candidate_orientations=array([[0., 2., 0.]]), distances=array([0.99999976], dtype=float32), mean_orientation=None, success=False, similar_indices=array([0])),
 OrientationResult(query_vector=array([0., 0., 0., ..., 0., 0., 0.], shape=(16384,)), best_orientation=array([0., 3., 0.]), candidate_orientations=array(